In [2]:
"""
    Linear mixed effects modeling for CPIG items
"""
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

In [17]:
df = pd.read_csv("/home/aml7990/Code/creativity-item-generation/item_evaluation/linear_regression/Summary_CPS_trials.csv")
# df = df[["Item Gen Model", "Item Response Gen Model", "Shot Selection Method", "Response Prompt Type", "Mean Originality - Round 5"]]
df.dropna(inplace=True)
df = pd.melt(df, id_vars=["Item Gen Model", "Item Response Gen Model", "Shot Selection Method", "Response Prompt Type"], var_name="Round", value_name="Mean Originality")
df.rename(columns={
    "Item Gen Model": "IGM",
    "Item Response Gen Model": "IRGM",
    "Shot Selection Method": "SSM",
    "Response Prompt Type": "RPT",
    "Mean Originality": "Originality"
}, inplace=True)
df

,IGM,IRGM,SSM,RPT,Round,Originality
0,meta-llama/Llama-3.1-70B-Instruct,meta-llama/Llama-3.1-8B-Instruct,CS,Baseline,Mean Originality - Round 1,1.419
1,meta-llama/Llama-3.1-70B-Instruct,meta-llama/Llama-3.1-8B-Instruct,CS,Baseline,Mean Originality - Round 1,1.414
2,meta-llama/Llama-3.1-70B-Instruct,meta-llama/Llama-3.1-8B-Instruct,CS,Baseline,Mean Originality - Round 1,1.355
3,meta-llama/Llama-3.1-70B-Instruct,meta-llama/Llama-3.1-8B-Instruct,CS,Demographic,Mean Originality - Round 1,1.305
4,meta-llama/Llama-3.1-70B-Instruct,meta-llama/Llama-3.1-8B-Instruct,CS,Demographic,Mean Originality - Round 1,1.398
...,...,...,...,...,...,...
105,gpt-4o-mini,meta-llama/Llama-3.1-8B-Instruct,Random,Demographic,Mean Originality - Round 5,1.318
106,gpt-4o-mini,meta-llama/Llama-3.1-8B-Instruct,Random,Demographic,Mean Originality - Round 5,1.436
107,gpt-4o-mini,meta-llama/Llama-3.1-8B-Instruct,Random,Baseline,Mean Originality - Round 5,1.472
108,gpt-4o-mini,meta-llama/Llama-3.1-8B-Instruct,Random,Baseline,Mean Originality - Round 5,1.450


In [19]:
# The simpliest possible model: simply treat all variables as independent
model1 = smf.ols(formula="Originality ~ C(IGM) + C(IRGM) + C(SSM) + C(RPT) + C(Round)", data=df)
res = model1.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Originality   R-squared:                       0.853
Model:                            OLS   Adj. R-squared:                  0.840
Method:                 Least Squares   F-statistic:                     64.60
Date:                Thu, 12 Dec 2024   Prob (F-statistic):           1.10e-37
Time:                        14:20:59   Log-Likelihood:                 110.16
No. Observations:                 110   AIC:                            -200.3
Df Residuals:                     100   BIC:                            -173.3
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
===================================================================================================================
                                                      coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
Intercept                                           0.7707      0.027     28.097      0.000       0.716       0.825
C(IGM)[T.gpt-4o-mini]                               0.1539      0.021      7.277      0.000       0.112       0.196
C(IGM)[T.meta-llama/Llama-3.1-70B-Instruct]         0.1003      0.031      3.227      0.002       0.039       0.162
C(IGM)[T.mistralai/Mistral-Large-Instruct-2407]     0.1268      0.031      4.080      0.000       0.065       0.188
C(IRGM)[T.meta-llama/Llama-3.1-8B-Instruct]         0.4949      0.021     23.399      0.000       0.453       0.537
C(IRGM)[T.meta-llama/Llama-3.2-3B-Instruct]         0.3149      0.031     10.133      0.000       0.253       0.377
C(SSM)[T.Greedy]                                   -0.0135      0.056     -0.243      0.809      -0.124       0.097
C(SSM)[T.Random]                                   -0.1083      0.025     -4.277      0.000      -0.158      -0.058
C(RPT)[T.Demographic]                               0.0189      0.022      0.860      0.392      -0.025       0.062
C(RPT)[T.Psychometric]                              0.1944      0.022      8.848      0.000       0.151       0.238
C(Round)[T.Mean Originality - Round 5]              0.2431      0.018     13.674      0.000       0.208       0.278
==============================================================================
Omnibus:                        0.273   Durbin-Watson:                   1.239
Prob(Omnibus):                  0.872   Jarque-Bera (JB):                0.333
Skew:                          -0.115   Prob(JB):                        0.846
Kurtosis:                       2.860   Cond. No.                     6.19e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.76e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [20]:
# add interactions between generator and responder
# the nas worry me in these models, I think its better to fit separate models that don't have them
model2 = smf.ols(formula="Originality ~ C(IGM) : C(IRGM) + C(SSM) + C(RPT) + C(Round)", data=df)
res = model2.fit()
res.summary()

/home/aml7990/miniconda3/envs/AIG-CUDA-12.0/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Originality   R-squared:                       0.853
Model:                            OLS   Adj. R-squared:                  0.840
Method:                 Least Squares   F-statistic:                     64.60
Date:                Thu, 12 Dec 2024   Prob (F-statistic):           1.10e-37
Time:                        14:35:35   Log-Likelihood:                 110.16
No. Observations:                 110   AIC:                            -200.3
Df Residuals:                     100   BIC:                            -173.3
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=============================================================================================================================================================
                                                                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                     0.8028      0.015     55.253      0.000       0.774       0.832
C(SSM)[T.Greedy]                                                                              0.0784      0.046      1.706      0.091      -0.013       0.170
C(SSM)[T.Random]                                                                             -0.0163      0.028     -0.590      0.557      -0.071       0.039
C(RPT)[T.Demographic]                                                                         0.0189      0.022      0.860      0.392      -0.025       0.062
C(RPT)[T.Psychometric]                                                                        0.1944      0.022      8.848      0.000       0.151       0.238
C(Round)[T.Mean Originality - Round 5]                                                        0.2431      0.018     13.674      0.000       0.208       0.278
C(IRGM)[T.meta-llama/Llama-3.1-8B-Instruct]                                                   0.4628      0.018     26.064      0.000       0.428       0.498
C(IRGM)[T.meta-llama/Llama-3.2-3B-Instruct]                                                   0.2183      0.011     20.511      0.000       0.197       0.239
C(IGM)[T.gpt-4o-mini]:C(IRGM)[lmsys/vicuna-7b-v1.5]                                           0.1218      0.018      6.858      0.000       0.087       0.157
C(IGM)[T.meta-llama/Llama-3.1-70B-Instruct]:C(IRGM)[lmsys/vicuna-7b-v1.5]                   2.34e-17   5.98e-18      3.916      0.000    1.15e-17    3.53e-17
C(IGM)[T.mistralai/Mistral-Large-Instruct-2407]:C(IRGM)[lmsys/vicuna-7b-v1.5]             -2.002e-17   7.52e-18     -2.663      0.009   -3.49e-17    -5.1e-18
C(IGM)[T.gpt-4o-mini]:C(IRGM)[meta-llama/Llama-3.1-8B-Instruct]                               0.0620      0.028      2.236      0.028       0.007       0.117
C(IGM)[T.meta-llama/Llama-3.1-70B-Instruct]:C(IRGM)[meta-llama/Llama-3.1-8B-Instruct]         0.1003      0.031      3.227      0.002       0.039       0.162
C(IGM)[T.mistralai/Mistral-Large-Instruct-2407]:C(IRGM)[meta-llama/Llama-3.1-8B-Instruct]     0.1268      0.031      4.080      0.000       0.065       0.188
C(IGM)[T.gpt-4o-mini]:C(IRGM)[meta-llama/Llama-3.2-3B-Instruct]                               0.2183      0.011     20.511      0.000       0.197       0.239
C(IGM)[T.meta-llama/Llama-3.1-70B-Instruct]:C(IRGM)[meta-llama/Llama-3.2-3B-Instruct]              0          0        nan        nan           0           0
C(IGM)[T.mistralai/Mistral-Large-Instruct-2407]:C(IRGM)[meta-llama/Llama-3.2-3B-I

In [23]:
# interaction between generator and shot selection method
model3 = smf.ols(formula="Originality ~ C(IGM) : C(SSM)  + C(IRGM)  + C(RPT) + C(Round)", data=df)
res = model3.fit()
res.summary()

/home/aml7990/miniconda3/envs/AIG-CUDA-12.0/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Originality   R-squared:                       0.853
Model:                            OLS   Adj. R-squared:                  0.840
Method:                 Least Squares   F-statistic:                     64.60
Date:                Thu, 12 Dec 2024   Prob (F-statistic):           1.10e-37
Time:                        14:39:28   Log-Likelihood:                 110.16
No. Observations:                 110   AIC:                            -200.3
Df Residuals:                     100   BIC:                            -173.3
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==================================================================================================================================
                                                                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          0.7300      0.015     49.911      0.000       0.701       0.759
C(IRGM)[T.meta-llama/Llama-3.1-8B-Instruct]                        0.5355      0.017     31.184      0.000       0.501       0.570
C(IRGM)[T.meta-llama/Llama-3.2-3B-Instruct]                        0.3149      0.031     10.133      0.000       0.253       0.377
C(RPT)[T.Demographic]                                              0.0189      0.022      0.860      0.392      -0.025       0.062
C(RPT)[T.Psychometric]                                             0.1944      0.022      8.848      0.000       0.151       0.238
C(Round)[T.Mean Originality - Round 5]                             0.2431      0.018     13.674      0.000       0.208       0.278
C(SSM)[T.Greedy]                                                   0.0702      0.035      1.987      0.050       0.000       0.140
C(SSM)[T.Random]                                                   0.0228      0.016      1.470      0.145      -0.008       0.054
C(IGM)[T.gpt-4o-mini]:C(SSM)[CS]                                   0.1945      0.017     11.327      0.000       0.160       0.229
C(IGM)[T.meta-llama/Llama-3.1-70B-Instruct]:C(SSM)[CS]             0.1003      0.031      3.227      0.002       0.039       0.162
C(IGM)[T.mistralai/Mistral-Large-Instruct-2407]:C(SSM)[CS]         0.1268      0.031      4.080      0.000       0.065       0.188
C(IGM)[T.gpt-4o-mini]:C(SSM)[Greedy]                               0.0702      0.035      1.987      0.050       0.000       0.140
C(IGM)[T.meta-llama/Llama-3.1-70B-Instruct]:C(SSM)[Greedy]              0          0        nan        nan           0           0
C(IGM)[T.mistralai/Mistral-Large-Instruct-2407]:C(SSM)[Greedy]          0          0        nan        nan           0           0
C(IGM)[T.gpt-4o-mini]:C(SSM)[Random]                               0.0228      0.016      1.470      0.145      -0.008       0.054
C(IGM)[T.meta-llama/Llama-3.1-70B-Instruct]:C(SSM)[Random]              0          0        nan        nan           0           0
C(IGM)[T.mistralai/Mistral-Large-Instruct-2407]:C(SSM)[Random]          0          0        nan        nan           0           0
==============================================================================
Omnibus:                        0.273   Durbin-Watson:                   1.239
Prob(Omnibus):                  0.872   Jarque-Bera (JB):                0.333
Skew:                          -0.115   Prob(JB):                        0.846
Kurtosis:                       2.860   Cond. No.                          inf
====================================================================

In [25]:
# interaction between prompt type and round
model4 = smf.ols(formula="Originality ~ C(IGM) : C(RPT) + C(IGM) + C(RPT) + C(Round) + C(IRGM) + C(SSM)", data=df)
res = model4.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Originality   R-squared:                       0.865
Model:                            OLS   Adj. R-squared:                  0.844
Method:                 Least Squares   F-statistic:                     40.22
Date:                Thu, 12 Dec 2024   Prob (F-statistic):           3.22e-34
Time:                        14:48:29   Log-Likelihood:                 114.83
No. Observations:                 110   AIC:                            -197.7
Df Residuals:                      94   BIC:                            -154.4
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================================
                                                                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                  0.7843      0.031     25.208      0.000       0.723       0.846
C(IGM)[T.gpt-4o-mini]                                                      0.1430      0.030      4.794      0.000       0.084       0.202
C(IGM)[T.meta-llama/Llama-3.1-70B-Instruct]                                0.1087      0.053      2.042      0.044       0.003       0.214
C(IGM)[T.mistralai/Mistral-Large-Instruct-2407]                            0.0475      0.053      0.893      0.374      -0.058       0.153
C(RPT)[T.Demographic]                                                     -0.0503      0.053     -0.946      0.347      -0.156       0.055
C(RPT)[T.Psychometric]                                                     0.2010      0.053      3.778      0.000       0.095       0.307
C(Round)[T.Mean Originality - Round 5]                                     0.2431      0.018     13.832      0.000       0.208       0.278
C(IRGM)[T.meta-llama/Llama-3.1-8B-Instruct]                                0.5022      0.025     19.873      0.000       0.452       0.552
C(IRGM)[T.meta-llama/Llama-3.2-3B-Instruct]                                0.3149      0.031     10.250      0.000       0.254       0.376
C(SSM)[T.Greedy]                                                          -0.0235      0.056     -0.419      0.676      -0.135       0.088
C(SSM)[T.Random]                                                          -0.1155      0.029     -4.016      0.000      -0.173      -0.058
C(IGM)[T.gpt-4o-mini]:C(RPT)[T.Demographic]                                0.0792      0.061      1.289      0.201      -0.043       0.201
C(IGM)[T.meta-llama/Llama-3.1-70B-Instruct]:C(RPT)[T.Demographic]          0.0107      0.075      0.142      0.888      -0.139       0.160
C(IGM)[T.mistralai/Mistral-Large-Instruct-2407]:C(RPT)[T.Demographic]      0.1672      0.075      2.222      0.029       0.018       0.317
C(IGM)[T.gpt-4o-mini]:C(RPT)[T.Psychometric]                              -0.0248      0.061     -0.403      0.688      -0.147       0.097
C(IGM)[T.meta-llama/Llama-3.1-70B-Instruct]:C(RPT)[T.Psychometric]        -0.0358      0.075     -0.476      0.635      -0.185       0.114
C(IGM)[T.mistralai/Mistral-Large-Instruct-2407]:C(RPT)[T.Psychometric]     0.0707      0.075      0.939      0.350      -0.079       0.220
==============================================================================
Omnibus:                        1.754   Durbin-Watson:                   1.219
Prob(Omnibus):                  0.416   Jarque-Bera (JB):                1.228
Skew:                          -0.152   Prob(JB):                 

In [27]:
# likelihood ratio test TODO: confirm this is accurate
from scipy.stats import chi2
np.random.seed(0)

full_ll = 114.83 # model4
reduced_ll = 110.16 # model1
full_params = 15
reduced_params = 9

# Calculate the likelihood ratio
lr = 2 * (full_ll - reduced_ll)
degrees = full_params - reduced_params
p_value = 1 - chi2.cdf(lr, degrees)

print("Likelihood Ratio:", lr)
print("P-value:", p_value)

Likelihood Ratio: 9.340000000000003
P-value: 0.15534021266188536
